<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/ml_intro/blob/main/2_planificacion/redes_neuronales_geron/geron/10_chapter/pagina_491_edicion_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/ml_intro/blob/main/2_planificacion/redes_neuronales_geron/geron/10_chapter/pagina_491_edicion_3.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

Aquí tienes la traducción al español de las páginas **507 y 508** del libro *"Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow"* de Aurélien Géron, sección **"Building a Regression MLP Using the Sequential API"**:

---



### **Construyendo un MLP de Regresión Usando la API Secuencial**  
Volvamos al problema de la vivienda en California (*California housing*) y abordémoslo usando el mismo **MLP** (*Multi-Layer Perceptron*) que antes, con **3 capas ocultas de 50 neuronas cada una**, pero esta vez construyéndolo con Keras.  



Usar la **API Secuencial** para construir, entrenar, evaluar y utilizar un MLP de regresión es muy similar a lo que hicimos para clasificación. Las principales diferencias en el siguiente ejemplo son:  
1. **Capa de salida**: Tiene una sola neurona (ya que solo predecimos un valor) y **no usa función de activación**.  
2. **Función de pérdida**: Error cuadrático medio (*mean squared error*, MSE).  
3. **Métrica**: RMSE (*Root Mean Squared Error*).  
4. **Optimizador**: Usamos **Adam**, igual que `MLPRegressor` de Scikit-Learn.  

Además, en este ejemplo:  
- No necesitamos una capa `Flatten`.  
- Usamos una capa de **Normalización** como primera capa: hace lo mismo que `StandardScaler` de Scikit-Learn, pero debe ajustarse a los datos de entrenamiento con su método `adapt()` antes de llamar a `fit()`. (Keras tiene otras capas de preprocesamiento, que se cubrirán en el Capítulo 13).  

#### **Código Ejemplo**:  
```python
import tensorflow as tf

# Fijar semilla para reproducibilidad
tf.random.set_seed(42)

# Capa de Normalización (equivalente a StandardScaler)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])

# Modelo Secuencial
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)  # Sin activación para regresión
])

# Optimizador y compilación
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

# Ajustar normalización y entrenar
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

# Evaluación y predicción
mse_test, rmse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)
```

#### **Nota Importante**:  
La capa de `Normalization` aprende las **medias y desviaciones estándar** de las características al llamar a `adapt()`. Sin embargo, en el resumen del modelo, estos parámetros aparecen como **no entrenables** (*non-trainable*), porque no son afectados por el descenso de gradiente.  

---



### **Ventajas y Limitaciones de la API Secuencial**  
Como ves, la **API Secuencial** es clara y sencilla. No obstante, aunque los modelos secuenciales son muy comunes, a veces es necesario construir redes con:  
- Topologías más complejas.  
- Múltiples entradas o salidas.  

Para estos casos, Keras ofrece la **API Funcional** (que se explicará más adelante).  

---



### **Traducción de Términos Clave**:  
| Inglés | Español |  
|--------|---------|  
| Hidden layers | Capas ocultas |  
| Output layer | Capa de salida |  
| Mean squared error (MSE) | Error cuadrático medio |  
| Root Mean Squared Error (RMSE) | Raíz del error cuadrático medio |  
| Adam optimizer | Optimizador Adam |  
| Flatten layer | Capa de aplanamiento |  
| Normalization layer | Capa de normalización |  
| Non-trainable parameters | Parámetros no entrenables |  



Esta traducción conserva el **tono técnico** del original y adapta los conceptos para hispanohablantes, manteniendo los términos clave en inglés entre paréntesis cuando es relevante. ¿Necesitas alguna aclaración adicional?

¡Excelente petición! Vamos a diseñar una **práctica de codificación paso a paso** basada en el texto de Géron, centrada en desarrollar un **MLP para regresión** con Keras (como en el ejemplo del libro), pero añadiendo desafíos para fortalecer tu **pensamiento computacional**. Aprenderás a:  

1. **Preprocesar datos** (normalización).  
2. **Construir y entrenar un modelo secuencial**.  
3. **Evaluar y ajustar hiperparámetros**.  
4. **Analizar resultados** (visualización de métricas).  

---

### **Práctica: MLP para Predecir Precios de Viviendas en California**  
**Dataset**: Usaremos el mismo dataset que Géron (*California Housing*), disponible en `sklearn.datasets`.

#### **Paso 1: Configuración del Entorno**  
```python
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Cargar datos
housing = fetch_california_housing()
X, y = housing.data, housing.target

# Dividir en train (70%), validación (15%), test (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
```

#### **Paso 2: Preprocesamiento con Normalización**  
Aquí compararemos dos enfoques:  
- **Normalización con Keras** (como en Géron).  
- **StandardScaler de Scikit-Learn** (para entender diferencias).  

```python
# Opción 1: Normalización con Keras
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
norm_layer.adapt(X_train)  # Ajuste a datos de entrenamiento

# Opción 2: StandardScaler de Scikit-Learn
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)
```

#### **Paso 3: Construcción del Modelo Secuencial**  
```python
def build_model(normalization_layer=None):
    model = tf.keras.Sequential()
    if normalization_layer:
        model.add(normalization_layer)  # Usar capa de Keras
    model.add(tf.keras.layers.Dense(50, activation="relu"))
    model.add(tf.keras.layers.Dense(50, activation="relu"))
    model.add(tf.keras.layers.Dense(50, activation="relu"))
    model.add(tf.keras.layers.Dense(1))  # Salida lineal para regresión
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(loss="mse", metrics=["RootMeanSquaredError"])
    return model

# Modelo con normalización de Keras
model_keras = build_model(norm_layer)
history_keras = model_keras.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

# Modelo con StandardScaler
model_scaler = build_model()
history_scaler = model_scaler.fit(X_train_scaled, y_train, epochs=20, validation_data=(X_valid_scaled, y_valid))
```

#### **Paso 4: Evaluación y Visualización**  
```python
# Función para graficar curvas de aprendizaje
def plot_loss(history, title):
    plt.plot(history.history["loss"], label="Train")
    plt.plot(history.history["val_loss"], label="Validation")
    plt.xlabel("Epochs")
    plt.ylabel("MSE")
    plt.title(title)
    plt.legend()
    plt.show()

plot_loss(history_keras, "Normalización con Keras")
plot_loss(history_scaler, "Normalización con StandardScaler")

# Evaluar en test
mse_keras, rmse_keras = model_keras.evaluate(X_test, y_test)
mse_scaler, rmse_scaler = model_scaler.evaluate(X_test_scaled, y_test)
print(f"Keras Normalization - Test RMSE: {rmse_keras:.4f}")
print(f"StandardScaler - Test RMSE: {rmse_scaler:.4f}")
```

#### **Paso 5: Desafíos para Pensamiento Computacional**  
1. **Experimenta con Arquitecturas**:  
   - ¿Qué pasa si cambias el número de neuronas (ej.: 30 en lugar de 50)?  
   - Prueba añadir una cuarta capa oculta.  

2. **Ajuste de Hiperparámetros**:  
   - Modifica el `learning_rate` del optimizador Adam (prueba 1e-2, 1e-4).  
   - Usa **early stopping** para evitar sobreajuste:  
     ```python
     early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
     history = model.fit(..., callbacks=[early_stopping])
     ```

3. **Compara Preprocesadores**:  
   - ¿Cuál método de normalización da mejores resultados? ¿Por qué?  

4. **Predicciones Cualitativas**:  
   - Imprime las primeras 5 predicciones del modelo y compáralas con los valores reales:  
     ```python
     y_pred = model_keras.predict(X_test[:5])
     print("Predicciones:", y_pred.flatten())
     print("Valores Reales:", y_test[:5])
     ```

---

### **Resultado Esperado**  
- Aprenderás a **debuggear modelos**: Si el RMSE es muy alto, revisa si los datos están bien normalizados.  
- Entenderás el impacto de **la arquitectura y el learning rate** en el entrenamiento.  
- Visualizarás el **sobreajuste** (si el loss de validación sube en epochs altas).  

---

### **Conclusión**  
Esta práctica refleja el **flujo de trabajo real en ML**:  
1. Preprocesar datos → 2. Construir modelo → 3. Entrenar/Ajustar → 4. Evaluar.  
**Tips adicionales**:  
- Usa `model.summary()` para ver la estructura del modelo.  
- Explora `tf.keras.utils.plot_model()` para visualizar la arquitectura.  

¿Quieres profundizar en algún paso en particular? Por ejemplo, ¿cómo implementar un **modelo con la API Funcional** para compararlo con el Secuencial?